In [9]:
import ccxt
import time
import concurrent.futures
from datetime import datetime, timezone

# Replace with your actual OKX API credentials
OKX_API_KEY = '48af790d-3250-44ec-9528-54d5cd1c1009'
OKX_API_SECRET = 'C91F9F5826FD2F5A8ADC91D945643AD9'
OKX_API_PASSWORD = 'OkX!?Secure789Trade2025?!'

# Connect to OKX
okx = ccxt.okx({
    'apiKey': OKX_API_KEY,
    'secret': OKX_API_SECRET,
    'password': OKX_API_PASSWORD,
    'enableRateLimit': True,
    'options': {'adjustForTimeDifference': True}
})

In [ ]:
# ⚙️ Config Settings
SPREAD_THRESHOLD = 0.01 / 100  # 0.1%
MIN_VOLUME = 50000  # Minimum 24h USDT volume
TRADING_FEE = 0.0007  # 0.07% fee per side
CHECK_INTERVAL = 5  # seconds

futures_markets_cache = []

# 📅 Timestamped logger
def log(msg):
    timestamp = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{timestamp}] {msg}")

# 🔎 Load tradable tokens (spot) and cache USDT futures
def get_tradable_tokens():
    try:
        markets = okx.load_markets()
        tradable_tokens = []
        global futures_markets_cache

        # ✅ Cache USDT-margined perpetual futures
        futures_markets_cache = [
            symbol for symbol, market in markets.items()
            if market.get('type') == 'swap' and market.get('linear') and market.get('quote') == 'USDT'
        ]
        log(f"📦 Cached {len(futures_markets_cache)} futures markets")
        print("🔍 Sample futures markets:", futures_markets_cache[:5])

        for symbol, market in markets.items():
            if market.get('spot') and '/USDT' in symbol:
                try:
                    ticker = okx.fetch_ticker(symbol)
                    volume = ticker.get('quoteVolume', 0)
                    if volume >= MIN_VOLUME:
                        tradable_tokens.append(symbol)
                except Exception:
                    continue

        log(f"✅ Found {len(tradable_tokens)} tradable tokens on OKX")
        return tradable_tokens
    except Exception as e:
        log(f"❌ Error loading markets: {e}")
        return []

# 📈 Fetch latest price
def get_price(symbol):
    try:
        ticker = okx.fetch_ticker(symbol)
        return ticker.get("last")
    except Exception:
        return None

# 🧵 Multithreaded price fetcher
def fetch_prices_concurrently(symbols):
    prices = {}
    def fetch(symbol):
        prices[symbol] = get_price(symbol)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(fetch, symbols)
    return prices

# 🔗 Match futures for a spot symbol
def find_futures_symbol(spot_symbol):
    base = spot_symbol.split('/')[0]
    future_symbol = f"{base}/USDT:USDT"
    return future_symbol if future_symbol in futures_markets_cache else None

# 💰 Check for profitable arbitrage
def check_arbitrage(symbol, spot_prices, futures_prices, profitable_opps):
    spot_price = spot_prices.get(symbol)
    futures_symbol = find_futures_symbol(symbol)
    if not futures_symbol:
        return
    futures_price = futures_prices.get(futures_symbol)
    if not spot_price or not futures_price:
        return

    spread = (futures_price - spot_price) / spot_price
    net_profit = spread - (TRADING_FEE * 2)

    if net_profit >= SPREAD_THRESHOLD:
        profitable_opps.append({
            'symbol': symbol,
            'spot_price': spot_price,
            'futures_price': futures_price,
            'spread': spread,
            'net_profit': net_profit
        })

# 🚀 Main arbitrage scanner loop
def main():
    while True:
        try:
            tokens = get_tradable_tokens()
            if not tokens:
                time.sleep(10)
                continue

            matched = []
            unmatched = []

            for token in tokens:
                if find_futures_symbol(token):
                    matched.append(token)
                else:
                    unmatched.append(token)

            log(f"🔗 Matched: {len(matched)} tokens with futures")
            log(f"❌ Unmatched: {len(unmatched)} tokens with no futures pair")

            if unmatched:
                print("⚠️ Unmatched Tokens:")
                for u in unmatched[:10]:
                    print(f"   - {u}")

            spot_prices = fetch_prices_concurrently(matched)
            futures_symbols = [find_futures_symbol(t) for t in matched]
            futures_prices = fetch_prices_concurrently(futures_symbols)

            profitable_opps = []
            with concurrent.futures.ThreadPoolExecutor() as executor:
                tasks = [
                    executor.submit(check_arbitrage, token, spot_prices, futures_prices, profitable_opps)
                    for token in matched
                ]
                concurrent.futures.wait(tasks)

            if profitable_opps:
                print("\n💰 PROFITABLE ARBITRAGE OPPORTUNITIES FOUND:\n")
                profitable_opps.sort(key=lambda x: x['net_profit'], reverse=True)
                for opp in profitable_opps:
                    print(f"🔍 Arbitrage Opportunity on OKX: {opp['symbol']}")
                    print(f"✅ Spot Price: {opp['spot_price']:.5f} USDT")
                    print(f"✅ Futures Price: {opp['futures_price']:.5f} USDT")
                    print(f"✅ Spread: {opp['spread']:.4%}")
                    print(f"✅ Net Profit After Fees: {opp['net_profit']:.4%}")
                    print(f"⚡ Execute arbitrage on OKX")
                    print("-" * 60)
            else:
                print("🕵️ No profitable arbitrage opportunities this round.")

            log("✅ OKX scan complete. Waiting for next cycle...\n")
            time.sleep(CHECK_INTERVAL)

        except Exception as e:
            log(f"❌ Main loop error: {type(e).__name__}: {e}")
            time.sleep(10)

if __name__ == '__main__':
    main()

[2025-03-25 07:36:27] 📦 Cached 243 futures markets
🔍 Sample futures markets: ['BTC/USDT:USDT', 'ETH/USDT:USDT', 'SOL/USDT:USDT', 'TON/USDT:USDT', 'DOGE/USDT:USDT']
[2025-03-25 07:37:00] ✅ Found 287 tradable tokens on OKX
[2025-03-25 07:37:00] 🔗 Matched: 215 tokens with futures
[2025-03-25 07:37:00] ❌ Unmatched: 72 tokens with no futures pair
⚠️ Unmatched Tokens:
   - OKB/USDT
   - ACA/USDT
   - ALCX/USDT
   - ASTR/USDT
   - BABYDOGE/USDT
   - BANANA/USDT
   - BETH/USDT
   - BONE/USDT
   - BTT/USDT
   - CELR/USDT
🕵️ No profitable arbitrage opportunities this round.
[2025-03-25 07:37:04] ✅ OKX scan complete. Waiting for next cycle...

[2025-03-25 07:37:09] 📦 Cached 243 futures markets
🔍 Sample futures markets: ['BTC/USDT:USDT', 'ETH/USDT:USDT', 'SOL/USDT:USDT', 'TON/USDT:USDT', 'DOGE/USDT:USDT']
[2025-03-25 07:37:42] ✅ Found 287 tradable tokens on OKX
[2025-03-25 07:37:42] 🔗 Matched: 215 tokens with futures
[2025-03-25 07:37:42] ❌ Unmatched: 72 tokens with no futures pair
⚠️ Unmatched To